# MOGPTK

## 1) Install Required Packages

In [ ]:
import sys
!{sys.executable} -m pip install tensorflow-probability==0.8.0 mogptk

## 2) Load Libraries

In [ ]:
import sys
sys.path.insert(0, '../')
%reload_ext autoreload
%autoreload 2

import mogptk
#import gpflow
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import datetime
import scipy as sp
import math
import tensorflow as tf

# plot config
sns.set_context('paper', font_scale=1.4)
sns.set_style('ticks')
plt.rcParams['figure.figsize'] = (10, 5)

## 3) Define Functions

### Basic FrameWork

In [ ]:
from scipy.special import iv
from scipy.optimize import fsolve

def vonmises_density(x,mu,kappa):
    """
    Calculate the von Mises density for a series x (a 1D numpy.array).
    Input : 
        x : a 1D numpy.array of size L
        mu : a 1D numpy.array of size n, the mean of the von Mises distributions
        kappa : a 1D numpy.array of size n, the dispersion of the von Mises distributions
    Output : 
        a (L x n) numpy array, L is the length of the series, and n is the size of the array containing the parameters. Each row of the output corresponds to a density
    """
    res = []
    for i in x:
        f = np.exp(kappa*np.cos(i-mu))
        n = 2*np.pi*iv(0,kappa)
        res.append(f/n)
    return(np.array(res))

def vonmises_pdfit(series):
    """
    Calculate the estimator of the mean and deviation of a sample, for a von Mises distribution
    Input : 
        series : a 1D numpy.array
    Output : 
        the estimators of the parameters mu and kappa of a von Mises distribution, in an list [mu, kappa]
    See https://en.wikipedia.org/wiki/Von_Mises_distribution 
    for more details on the von Mises distribution
    """
    s0 = np.mean(np.sin(series))
    c0 = np.mean(np.cos(series))
    mu = np.arctan2(s0,c0)
    var = 1-np.sqrt(s0**2+c0**2)
    k = lambda kappa: 1-iv(1,kappa)/iv(0,kappa)-var
    kappa = fsolve(k, 0.0)[0]
    return([mu,kappa])

In [ ]:
from scipy.stats import vonmises
# Here is the function to define KL divergence
# Note: the samplepdf and observationpdf is the probability density function, not the sample count.

from scipy.stats import entropy

def kldiver(samplepdf, observationpdf):
    print("\nIndividual Entropy\n")
    print(entropy(samplepdf))
    print(entropy(observationpdf))

    print("\nPairwise Kullback Leibler divergence\n")
    firstkl = entropy(samplepdf, qk=observationpdf)
    secondkl = entropy(observationpdf, qk=samplepdf)
    print(firstkl)
    print(secondkl)
    return (firstkl,secondkl)

In [ ]:
from scipy import stats
# Here is the function to define KL divergence
# Note: the samplepdf and observationpdf is the probability density function, not the sample count.

from scipy.stats import entropy

def kldiver2(samplepdf, observationpdf):
    #print("\nIndividual Entropy\n")
    #print(entropy(samplepdf))
    #print(entropy(observationpdf))

    #print("\nPairwise Kullback Leibler divergence\n")
    firstkl = entropy(samplepdf, qk=observationpdf)
    secondkl = entropy(observationpdf, qk=samplepdf)
    #print(firstkl)
    #print(secondkl)
    #return (firstkl,secondkl)
    return (firstkl+secondkl)/2

In [ ]:
from scipy import interpolate

def invcdf(vals, pdf, num_sam):
    # Normalize
    normalize = pdf/np.sum(pdf)
    p = np.cumsum(normalize)
    # define inverse function
    inv_cdf = interpolate.interp1d(p,vals,bounds_error=False, fill_value = (-math.pi, math.pi))
    # get number of data
    r = np.random.rand(num_sam)
    # get sample
    sample = inv_cdf(r)
    return sample

In [ ]:
# Here is the function to define the MSE
def countdiff(bins, sample, observation):
    if len(observation) != len(sample):
        print("Please generate the same length data")
    length = len(observation)
    sumup = 0
    sumup2 = 0
    # counts and divisions in the real data
    count,division = np.histogram(listall,range=(-math.pi,math.pi),bins=bins)
    # change the sample to numpy array, note: if it is already np array, comment it out
    s = sample
    # compute MSE
    for j in range(len(count)):
        modelCount = s[(division[j] < s) & (s < division[j+1])].size
        sumup += np.square(count[j] - modelCount)
        sumup2+= np.abs(count[j] - modelCount)
    mse = sumup/bins
    mae = sumup2/bins
    print('The MSE is ' + str(mse))
    print('The MAE is ' + str(mae))
    return mse, mae 

In [ ]:
# Here is the function to define the MSE
def countdiffmse(bins, sample, observation):
    length = len(observation)
    sumup = 0
    sumup2 = 0
    # counts and divisions in the real data
    count,division = np.histogram(listall,range=(-math.pi,math.pi),bins=bins)
    # change the sample to numpy array, note: if it is already np array, comment it out
    s = sample
    # compute MSE
    for j in range(len(count)):
        modelCount = s[(division[j] < s) & (s < division[j+1])].size
        sumup += np.square(count[j] - modelCount)
        sumup2+= np.abs(count[j] - modelCount)
    mse = sumup/bins
    mae = sumup2/bins
    return mse

In [ ]:
# Here is the function to define the MAE
def countdiffmae(bins, sample, observation):
    length = len(observation)
    sumup = 0
    sumup2 = 0
    # counts and divisions in the real data
    count,division = np.histogram(listall,range=(-math.pi,math.pi),bins=bins)
    # change the sample to numpy array, note: if it is already np array, comment it out
    s = sample
    # compute MSE
    for j in range(len(count)):
        modelCount = s[(division[j] < s) & (s < division[j+1])].size
        sumup += np.square(count[j] - modelCount)
        sumup2+= np.abs(count[j] - modelCount)
    mse = sumup/bins
    mae = sumup2/bins
    return mae

# Data

### Applying the data

In [ ]:
df = pd.read_csv("/home/idies/workspace/Storage/Genius/TRF/GPTK-SUB/June6th386users.csv")
df = df.drop(columns=["Unnamed: 0"])
df = df.drop(columns=["Unnamed: 0.1"])
df.head()

#### Convert Time String to make sure it will be in format (HH:MM:SS)

In [ ]:
start = []
end = []
for i in range(len(df['start_time'])):
    if len(df['start_time'][i]) < 8:
        start.append('0' + df['start_time'][i])
    else:
        start.append(df['start_time'][i])
    
    if len(df['end_time'][i]) < 8:
        end.append('0' + df['end_time'][i])
    else:
        end.append(df['end_time'][i])
df['start_time'] = start
df['end_time'] = end

#### Convert time string in integer value 0 - 24

In [ ]:
convert_start = []
convert_end = []
for i in range(0,len(df["end_time"])):
    convert_start.append(int(df["start_time"][i][0:2]) + int(df["start_time"][i][3:5])/60 + int(df["start_time"][i][6:])/3600)
    convert_end.append(int(df["end_time"][i][0:2]) + int(df["end_time"][i][3:5])/60 + int(df["end_time"][i][6:])/3600)
df["starthour"] = convert_start
df["endhour"] = convert_end

In [ ]:
df3 = df[(df["name.s"] == "Sleep")]
wake = df3["endhour"] * math.pi /12 - math.pi
sleep = df3["starthour"] * math.pi /12 - math.pi
df2 = df[(df["name.s"] == "Food")]
food = df2["starthour"] * math.pi /12 - math.pi
listall = list(wake) + list(sleep) + list(food)
wake_val = vonmises_pdfit(wake)
sleep_val = vonmises_pdfit(sleep)

#### Generate X and Y variable for MOGPTK

In [ ]:
# Create toy data set.
n = 1000
x = np.linspace(-math.pi, math.pi, n)
noise = 0.05

# Draw functions depending on each other in complicated ways.
# Wake
f1a = vonmises_density(x,wake_val[0],wake_val[1])
a1 = vonmises.rvs(wake_val[1],wake_val[0], size = 200)
# Food

food_val_a1 = vonmises_pdfit(list(food) +list(a1))
f2a = vonmises_density(x,food_val_a1[0],food_val_a1[1])
a2 = vonmises.rvs(food_val_a1[1],food_val_a1[0], size = 200)

# Sleep 
sleep_val_b = vonmises_pdfit(list(sleep) + list(a2))
f3a = vonmises_density(x,sleep_val_b[0],sleep_val_b[1])

f = np.stack((f1a, f2a, f3a), axis=0).T


# Add noise and subsample.
y = f + noise * np.random.randn(n,3)
x, y = x[::8], y[::8]

data1 = mogptk.Data(x,y[:,0],name ='Wake Up Time')
data2 = mogptk.Data(x,y[:,1],name ='Food Time')
data3 = mogptk.Data(x,y[:,2],name ='Sleep Time')
dataset = mogptk.DataSet(data1,data2,data3)

In [ ]:
for channel in dataset:
    channel.transform(mogptk.TransformDetrend(degree=1))
    channel.transform(mogptk.TransformWhiten())
dataset.plot();

#### Applying the MOSM method ( multi-output spectral mixture)

In [ ]:

model_MOSM = mogptk.MOSM(dataset, Q=2)

# initialize parameters of kernel
model_MOSM.init_parameters()
model_MOSM.print_parameters()

In [ ]:
# plot the prediction with untrained model
x_pred = [x for i in range(len(dataset))]
model_MOSM.predict(x_pred);

model_MOSM.plot_prediction(title='Untrained model for MOSM');

In [ ]:
model_MOSM.train(
    method='L-BFGS-B',
    tol=1e-60,
    maxiter=500,
    verbose=True)

In [ ]:
model_MOSM.predict(x_pred);

#model.plot_prediction(title='Trained model');
#model.predict(x_pred)
model_MOSM.plot_prediction(title='Trained model for MOSM');

#### Applying the CSM method (Cross Spectral Mixture)

In [ ]:

model_csm = mogptk.CSM(dataset, Q=2)

# initialize parameters of kernel
model_csm.init_parameters()
model_csm.print_parameters()

In [ ]:
# plot the prediction with untrained model
x_pred = [x for i in range(len(dataset))]
model_csm.predict(x_pred);

model_csm.plot_prediction(title='Untrained model for CSM');

In [ ]:
model_csm.train(
    method='L-BFGS-B',
    tol=1e-60,
    maxiter=500,
    verbose=True)

In [ ]:
model_csm.predict(x_pred);

#model.plot_prediction(title='Trained model');
#model.predict(x_pred)
model_csm.plot_prediction(title='Trained model for MOSM');

#### Applying the SM-LMC Method (Spectral Mixture - Linear Model of Coregionalization)

In [ ]:

model_smlmc = mogptk.SM_LMC(dataset, Q=2)

# initialize parameters of kernel
model_smlmc.init_parameters()
model_smlmc.print_parameters()

In [ ]:
# plot the prediction with untrained model
x_pred = [x for i in range(len(dataset))]
model_smlmc.predict(x_pred);

model_smlmc.plot_prediction(title='Untrained model for SM_LMC');

In [ ]:
model_smlmc.train(
    method='L-BFGS-B',
    tol=1e-60,
    maxiter=500,
    verbose=True)

In [ ]:
model_smlmc.predict(x_pred);

#model.plot_prediction(title='Trained model');
#model.predict(x_pred)
model_smlmc.plot_prediction(title='Trained model for SM_LMC');

### Combined all variable

In [ ]:
plt.plot(x,y[:,0],label = "Wake Up Time")
plt.plot(x,y[:,1],label = "Meal Time")
plt.plot(x,y[:,2],label = "Sleep Time")
plt.legend()

In [ ]:
ya= np.zeros(len(y[:,0]))
for i in range(0, len(y[:,0])):
    ya[i] = (y[:,0][i] + y[:,1][i]  +y[:,2][i])/3
plt.plot(x,ya)

In [ ]:
datan = mogptk.Data(x, ya, name='Total')
dataset1 = mogptk.DataSet(datan)

#### Applying the MOSM method ( multi-output spectral mixture)

In [ ]:

model_mosm = mogptk.MOSM(dataset1, Q=2)

# initialize parameters of kernel
model_mosm.init_parameters()
model_mosm.print_parameters()

In [ ]:
# plot the prediction with untrained model
x_pred = [x for i in range(len(dataset1))]
model_mosm.predict(x_pred);

model_mosm.plot_prediction(title='Untrained model for MOSM');

In [ ]:
model_mosm.train(
    method='L-BFGS-B',
    tol=1e-60,
    maxiter=500,
    verbose=True)

In [ ]:
model_mosm.predict(x_pred);

#model.plot_prediction(title='Trained model');
#model.predict(x_pred)
model_mosm.plot_prediction(title='Trained model for MOSM');

#### Applying the CSM method (Cross Spectral Mixture)

In [ ]:

model_csm = mogptk.CSM(dataset1, Q=2)

# initialize parameters of kernel
model_csm.init_parameters()
model_csm.print_parameters()

In [ ]:
# plot the prediction with untrained model
x_pred = [x for i in range(len(dataset1))]
model_csm.predict(x_pred);

model_csm.plot_prediction(title='Untrained model for CSM');

In [ ]:
model_csm.train(
    method='L-BFGS-B',
    tol=1e-60,
    maxiter=500,
    verbose=True)

In [ ]:
model_csm.predict(x_pred);

#model.plot_prediction(title='Trained model');
#model.predict(x_pred)
model_csm.plot_prediction(title='Trained model for CSM');

#### Applying the SM-LMC Method (Spectral Mixture - Linear Model of Coregionalization)

In [ ]:

model_smlmc = mogptk.SM_LMC(dataset1, Q=2)

# initialize parameters of kernel
model_smlmc.init_parameters()
model_smlmc.print_parameters()

In [ ]:
# plot the prediction with untrained model
x_pred = [x for i in range(len(dataset1))]
model_smlmc.predict(x_pred);

model_smlmc.plot_prediction(title='Untrained model for SM_LMC');

In [ ]:
model_smlmc.train(
    method='L-BFGS-B',
    tol=1e-60,
    maxiter=500,
    verbose=True)

In [ ]:
model_csm.predict(x_pred);

#model.plot_prediction(title='Trained model');
#model.predict(x_pred)
model_csm.plot_prediction(title='Trained model for SM_LMC');

#### Calculating the KL Divergence and MSE for this data

In [ ]:
w1=model_mosm.predict()
listall = list(wake) + list(sleep) + list(food)
wn = w1[1][0]

In [ ]:
sns.kdeplot(listall,label='Actual data based kernel desnsity estimation')
plt.plot(x, wn, label = 'Trained Data for MOGPTK')
plt.hist(listall,density=True,label='Actual Data')
plt.ylabel('Probability')
plt.xlabel('Data')
plt.legend()

In [ ]:
from scipy import stats
from scipy.stats import entropy
actual_density = stats.kde.gaussian_kde(listall)
actual_density = actual_density(x)
def converter(val):
    for i in range(0, len(val)):
        if (val[i] <=0):
            val[i] = 10E-300
    return val

In [ ]:
converter(wn);
print("\nIndividual Entropy\n")
print(entropy(wn, actual_density))

In [ ]:
papermse = []
papermae = []
for i in range(1000):
    samples = invcdf(x,wn, len(listall))
    papermse.append(countdiffmse(48,samples, listall))
    papermae.append(countdiffmae(48,samples, listall))

In [ ]:
meanmse = np.mean(papermse)
meanmae = np.mean(papermae)
lowersmse = np.percentile(papermse, 2.5)
uppersmse = np.percentile(papermse, 100 - 2.5)
lowersmae = np.percentile(papermae, 2.5)
uppersmae = np.percentile(papermae, 100 - 2.5)

print("MSE mean is " + str(meanmse))
print("MSE lower is " + str(lowersmse))
print("MSE upper is " + str(uppersmse))

print("MAE mean is " + str(meanmae))
print("MAE lower is " + str(lowersmae))
print("MAE upper is " + str(uppersmae))